In [1]:
from Tempo import Tempo
from datetime import timedelta, date, datetime
from IPython.display import clear_output

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
obj = Tempo()

In [7]:
end_date = date(2018, 8, 12)
start_date = date(2018, 2, 3)

In [8]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], single_date.strftime("%Y/%m/%d"))

2018-02-03
https://www.tempo.co/indeks/2018/02/03
Connection Error, but it's still trying...
https://www.tempo.co/indeks/2018/02/03
Connection Error, but it's still trying...
https://www.tempo.co/indeks/2018/02/03
Connection Error, but it's still trying...
https://www.tempo.co/indeks/2018/02/03
Connection Error, but it's still trying...
https://www.tempo.co/indeks/2018/02/03
Connection Error, but it's still trying...
https://www.tempo.co/indeks/2018/02/03
Connection Error, but it's still trying...
https://www.tempo.co/indeks/2018/02/03
Connection Error, but it's still trying...
https://www.tempo.co/indeks/2018/02/03
https://bisnis.tempo.co/read/1056921/ri-genjot-ekspor-nanas-dan-buah-naga-ke-cina-ini-alasannya
memasukkan berita id  1056921
Insert berita  RI Genjot Ekspor Nanas dan Buah Naga ke Cina, Ini Alasannya
masuk
https://otomotif.tempo.co/read/1056922/jeep-cherokee-2019-punya-segudang-keunggulan
memasukkan berita id  1056922
Insert berita  Jeep Cherokee 2019 Punya Segudang Keungg

memasukkan berita id  1056865
Insert berita  Piala Davisi: Perbandingan Peringkat Pemain Indonesia Vs Filipina
masuk
https://bisnis.tempo.co/read/1056830/investasi-asing-indonesia-rendah-dibanding-malaysia-dan-vietnam
memasukkan berita id  1056830
Insert berita  Investasi Asing Indonesia Rendah Dibanding Malaysia dan Vietnam
masuk
https://cantik.tempo.co/read/1056888/menu-sarapan-nikmat-dengan-resep-thai-banana-pancake
memasukkan berita id  1056888
Insert berita  Menu Sarapan Nikmat dengan Resep Thai Banana Pancake
masuk
https://bola.tempo.co/read/1056848/jadwal-liga-inggris-malam-nanti-ada-arsenal-vs-everton
memasukkan berita id  1056848
Insert berita  Jadwal Liga Inggris Malam Nanti, Ada Arsenal vs Everton
masuk
https://nasional.tempo.co/read/1056897/penganiayaan-guru-oleh-siswa-di-sampang-pelaku-boleh-ikut-un
memasukkan berita id  1056897
Insert berita  Penganiayaan Guru oleh Siswa di Sampang, Pelaku Boleh Ikut UN 
masuk
https://metro.tempo.co/read/1056874/reklamasi-kata-kadishub-so

UnboundLocalError: local variable 'response' referenced before assignment